# Activity 1: Energy-Minimized PHA oligomer

Before you proceed this workflow, I encourage you to follow the tutorials on ```00_tutorials/``` folder prior programming to have everything downloaded.

In this workflow, we will retrieve energy-minimized PHA oligomer 3D structure for molecular docking and dynamics.

Before you continue, please make sure you have already followed the instructions from ```00_tutorials/04_setting_vscode.md```.

Make sure you have already installed ```RDKit``` and ```Openbabel``` libraries. Otherwise, copy the following lines and paste it on the terminal.

1. ```conda activate vina```
3. ```sudo apt install openbabel```
2. ```conda install openbabel rdkit```


## 1. Import libraries

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem, rdChemReactions
from openbabel import pybel

## 2. Define monomer and polymerization reaction

Get the SMILES of monomer from PubChem: https://pubchem.ncbi.nlm.nih.gov/

In [ ]:
# Define monomer
monomer_smiles = "C[C@H](CC(=O)O)O"
# Convert monomer to RdKit-readable
monomer = Chem.MolFromSmiles(monomer_smiles)

In [ ]:
# Define polymer reaction (SMART structure)
# [Acid-C:1] + [Alcohol-O:4] -> [Ester]
rxn_smarts = '[C:1](=[O:2])[OH].[C:3][OH:4]>>[C:1](=[O:2])[O:4][C:3]'
# Convert polymer reaction into RdKit-readable
rxn = rdChemReactions.ReactionFromSmarts(rxn_smarts)

## 3. Build the oligomer (5 units)

In [ ]:
# Start with one monomer, then iteratively add others
oligomer = monomer
Chem.SanitizeMol(oligomer)
for i in range(4):  # Repeat to add 4 more units (total 5)
    products = rxn.RunReactants((oligomer, monomer))
    if products:
        oligomer = products[0][0]
        Chem.SanitizeMol(oligomer)
        #print(f"After reaction {i+1}: {Chem.MolToSmiles(oligomer)}")
    else:
        print("No reaction occurred")

Chem.SanitizeMol(oligomer)
print(f"Oligomer SMILES: {Chem.MolToSmiles(oligomer)}")
print(f"Number of Carbon atoms: {len(oligomer.GetSubstructMatches(Chem.MolFromSmarts('C')))}")

## 4. Energy-minimize the structure

In [ ]:
# Convert the SMILES into 3D structure
oligomer_3d = Chem.AddHs(oligomer)
AllChem.EmbedMolecule(oligomer_3d, AllChem.ETKDG())
AllChem.MMFFOptimizeMolecule(oligomer_3d)

In [ ]:
# Energy minimize with OpenBabel
mol_block = Chem.MolToMolBlock(oligomer_3d)
ob_mol = pybel.readstring("mol", mol_block)

In [ ]:
# Perform minimization using the MMFF94 force field (Other option: GAFF)
ob_mol.localopt(forcefield='mmff94', steps=500)

## 5. Export to PDB

In [ ]:
# Create file
output_file = "files/phb_oligomer.pdb"
ob_mol.write("pdb", output_file, overwrite=True)

print(f"Successfully exported minimized PHB oligomer to {output_file}")

END